In [1]:
# pip install shiba shiba-model evaluate datasets wandb arabert  accelerate -U nltk torchmetrics==0.3.2

In [2]:
from typing import Dict
import pandas as pd
import numpy as np
import torchmetrics
import torch
import transformers
from datasets import load_dataset, Dataset
from transformers import HfArgumentParser, Trainer, EvalPrediction

from shiba import ShibaForClassification, CodepointTokenizer
from training.helpers import DataArguments, get_base_shiba_state_dict,get_model_hyperparams, ShibaClassificationArgs, \
    ClassificationDataCollator

<h1> Choose the model

In [3]:
model_path = '../checkpoint-611960.pt'
seg_enable = True
bert_model_name = "aubmindlab/bert-base-arabertv02"
apply_farasa=False
file_save = 'Submit_64_05'

# Pre-Process data ( if needed)

<h1> Read the files

In [4]:
transformers.logging.set_verbosity_info()
device = "cuda"
parser = HfArgumentParser((ShibaClassificationArgs, DataArguments))

prediction_label = 'label'

df_train = pd.read_csv("data/IDAT_training_text.csv")
df_testOrignal = pd.read_csv("data/IDAT_test_text.csv")

categories = {idx: cat_name for idx, cat_name in enumerate(set(df_train[prediction_label]))}
id_by_category = {val: key for key, val in categories.items()}

print("categories : ", categories)
print("id_by_category : ", id_by_category)
df_train = Dataset.from_pandas(df_train)
df_test = Dataset.from_pandas(df_testOrignal)

categories :  {0: 0, 1: 1}
id_by_category :  {0: 0, 1: 1}


In [5]:
if seg_enable:
    from arabert.preprocess import ArabertPreprocessor

    arabert_prep = ArabertPreprocessor(model_name=bert_model_name,apply_farasa_segmentation=apply_farasa)
    # arabert_prep.preprocess()
    df_train = pd.DataFrame(df_train)
    df_test =  pd.DataFrame(df_test)

    df_train['text'] = df_train['text'].apply(arabert_prep.preprocess)
    df_test['text'] = df_test['text'].apply(arabert_prep.preprocess)
    df_train = Dataset.from_pandas(df_train)
    df_test = Dataset.from_pandas(df_test)

<h1> Check files count

In [6]:
len(df_train), len(df_test)

(4024, 1006)

<h1> Load the model

In [7]:
tokenizer = CodepointTokenizer()
model_hyperparams = {'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}
print(model_hyperparams)
model = ShibaForClassification(vocab_size=len(categories), **model_hyperparams)
data_collator = ClassificationDataCollator()
print('Loading and using base shiba states from', model_path)
checkpoint_state_dict = torch.load(model_path)
model.shiba_model.load_state_dict(get_base_shiba_state_dict(checkpoint_state_dict))

{'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loading and using base shiba states from ../checkpoint-611960.pt


<All keys matched successfully>

<h1>Input IDs Method

In [8]:
def process_example(example: Dict) -> Dict:
    return {
        'input_ids': tokenizer.encode(example['text'])['input_ids'][:model.config.max_length],
        'labels': id_by_category[example[prediction_label]]
    }
def process_exampleTemp(example: Dict) -> Dict:
    return {
        'input_ids': tokenizer.encode(example['text'])['input_ids'][:model.config.max_length],
        'labels': 0
    }

<h1> Compute Metrics

In [9]:
def compute_metrics(pred: EvalPrediction) -> Dict:
    try:
        # Convert predictions and labels to PyTorch tensors
        # label_probs = torch.tensor(pred.predictions)
        label_probs, embeddings = pred.predictions
        labels = torch.tensor(pred.label_ids)
        label_probs = torch.exp(torch.tensor(label_probs))  # undo the log in log softmax, get indices
        # # Compute accuracy
        # accuracy = torchmetrics.functional.accuracy(label_probs, labels, num_classes=len(categories))

        # Compute F1 score
        f1_score = torchmetrics.functional.f1(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute recall
        recall = torchmetrics.functional.recall(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute precision
        precision = torchmetrics.functional.precision(label_probs, labels, average='macro', num_classes=len(categories))

        # print("label_probs : ", label_probs, " labels : ", labels)

        metrics = {
            # 'accuracy': accuracy.item(),
            'f1_score': f1_score.item(),
            'recall': recall.item(),
            'precision': precision.item()
        }

        # print("metrics : ", metrics)
        # raise NotImplementedError

        return metrics
    except:
        print("pred : ", pred)
        print("pred.predictions : ", pred.predictions)
        print("label_probs : ", label_probs)
        print("label_probs.size : ", label_probs.size())
        print("labels : ", labels)
        print("labels.size() : ", labels.size())

        raise NotImplementedError

<h1> Fine-tune args

In [10]:
training_args = ShibaClassificationArgs(
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    data_seed=42,
    seed=42,
    do_eval=True,
    do_predict=True,
    do_train=True,
    dropout=0.5,
    eval_accumulation_steps=None,
    eval_delay=0,
    eval_steps=100,
    evaluation_strategy='steps',
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    output_dir="fine_result",
    prediction_loss_only=False,
    report_to=[],
    run_name="fine_result",
    save_strategy='no',
)

PyTorch: setting up devices


<h1> Setup the trainer

In [11]:
if seg_enable:
    compute_metrics = None

In [12]:
# print(all_data)
trainer = Trainer(model=model,
                args=training_args,
                data_collator=data_collator,
                train_dataset=df_train.map(process_example, remove_columns=list(df_train[0].keys())),
                eval_dataset=df_train.map(process_example, remove_columns=list(df_train[0].keys())),
                compute_metrics=compute_metrics,
                )

Map:   0%|          | 0/4024 [00:00<?, ? examples/s]

Map:   0%|          | 0/4024 [00:00<?, ? examples/s]

Using auto half precision backend


<h1> Train

In [13]:
training = trainer.train()

***** Running training *****
  Num examples = 4,024
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 630
  Number of trainable parameters = 120,767,234


Step,Training Loss,Validation Loss
100,0.570600,nan
200,0.405500,nan
300,0.266400,nan
400,0.203700,nan
500,0.132100,nan
600,0.089600,nan


***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4024
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




<h1> Prediction

In [14]:
pred = trainer.predict(df_test.map(process_exampleTemp, remove_columns=list(df_test[0].keys())))
df_testOrignal['prediction'] = [categories[x] for x in np.argmax(pred.predictions[0], axis=1)]

Parameter 'function'=<function process_exampleTemp at 0x7fd068de6b00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1006 [00:00<?, ? examples/s]

***** Running Prediction *****
  Num examples = 1006
  Batch size = 64


<h1> Saving

In [15]:
df_testOrignal[['id', 'prediction']].to_csv(file_save+'/irony.tsv', index=False ,sep="\t")
pd.read_csv(file_save+"/irony.tsv", sep="\t").head(3)

,id,prediction
0,0,0
1,1,0
2,2,1
